In [1]:
!nvidia-smi

Thu May 23 16:09:10 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.129.03             Driver Version: 535.129.03   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   42C    P8               9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [4]:
import os
HOME = os.getcwd()
print(HOME)

/kaggle/working


In [5]:
!pip install ultralytics==8.0.196

from IPython import display
display.clear_output()

import ultralytics
ultralytics.checks()

Ultralytics YOLOv8.0.196 🚀 Python-3.10.13 torch-2.1.2 CUDA:0 (Tesla T4, 15102MiB)
Setup complete ✅ (4 CPUs, 31.4 GB RAM, 5597.6/8062.4 GB disk)


In [12]:
from ultralytics import YOLO

from IPython.display import display, Image

In [13]:
!mkdir {HOME}/datasets
%cd {HOME}/datasets

/kaggle/working/datasets


In [14]:
!pip install roboflow

from roboflow import Roboflow
rf = Roboflow(api_key="1gBpoxVirzj0NHBuskuN")
project = rf.workspace("chaconneluofire").project("fire-smoke-kwgda")
version = project.version(4)
dataset = version.download("yolov8")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 74.9/74.9 kB 1.2 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.3/158.3 kB 3.0 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 178.7/178.7 kB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.8/58.8 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.1/49.1 MB 29.3 MB/s eta 0:00:00:00:0100:01
  Attempting uninstall: opencv-python-headless
    Found existing installation: opencv-python-headless 4.9.0.80
    Uninstalling opencv-python-headless-4.9.0.80:
      Successfully uninstalled opencv-python-headless-4.9.0.80
  Attempting uninstall: idna
    Found existing installation: idna 3.6
    Uninstalling idna-3.6:
      Successfully uninstalled idna-3.6
  Attempting uninstall: cycler
    Found existing installation: cycler 0.12.1
    Uninstalling cycler-0.12.1:
      Successfully uninstalled cycler-0.12.1
  Attempting uninstall: certifi
    Found existing install


Extracting Dataset Version Zip to Fire-Smoke-4 in yolov8:: 100%|██████████| 6646/6646 [00:00<00:00, 6669.81it/s]


In [15]:
%cd {HOME}

!yolo train model=yolov8s.pt data={dataset.location}/data.yaml epochs=100 imgsz=640 plots=True device=0,1 format=onnx batch=128 

/kaggle/working
100%|███████████████████████████████████████| 21.5M/21.5M [00:00<00:00, 151MB/s]
New https://pypi.org/project/ultralytics/8.2.20 available 😃 Update with 'pip install -U ultralytics'
Ultralytics YOLOv8.0.196 🚀 Python-3.10.13 torch-2.1.2 CUDA:0 (Tesla T4, 15102MiB)
                                                       CUDA:1 (Tesla T4, 15102MiB)
engine/trainer: task=detect, mode=train, model=yolov8s.pt, data=/kaggle/working/datasets/Fire-Smoke-4/data.yaml, epochs=100, patience=50, batch=128, imgsz=640, save=True, save_period=-1, cache=False, device=(0, 1), workers=8, project=None, name=None, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=Tru

In [16]:
!yolo export model=/kaggle/working/runs/detect/train/weights/best.pt format=onnx
!yolo export model=/kaggle/working/runs/detect/train/weights/last.pt format=onnx

Ultralytics YOLOv8.0.196 🚀 Python-3.10.13 torch-2.1.2 CPU (Intel Xeon 2.00GHz)
Model summary (fused): 168 layers, 11126358 parameters, 0 gradients, 28.4 GFLOPs

PyTorch: starting from '/kaggle/working/runs/detect/train/weights/best.pt' with input shape (1, 3, 640, 640) BCHW and output shape(s) (1, 6, 8400) (21.5 MB)

ONNX: starting export with onnx 1.16.0 opset 17...
ONNX: export success ✅ 1.3s, saved as '/kaggle/working/runs/detect/train/weights/best.onnx' (42.6 MB)

Export complete (3.9s)
Results saved to /kaggle/working/runs/detect/train/weights
Predict:         yolo predict task=detect model=/kaggle/working/runs/detect/train/weights/best.onnx imgsz=640  
Validate:        yolo val task=detect model=/kaggle/working/runs/detect/train/weights/best.onnx imgsz=640 data=/kaggle/working/datasets/Fire-Smoke-4/data.yaml  
Visualize:       https://netron.app
💡 Learn more at https://docs.ultralytics.com/modes/export
Ultralytics YOLOv8.0.196 🚀 Python-3.10.13 torch-2.1.2 CPU (Intel Xeon 2.00GHz)

In [17]:
!tar -czvf yolov8-b128-e100-img640-nosmoke.tar.gz /kaggle/working/runs/detect/train/

tar: Removing leading `/' from member names
/kaggle/working/runs/detect/train/
/kaggle/working/runs/detect/train/F1_curve.png
/kaggle/working/runs/detect/train/val_batch2_labels.jpg
/kaggle/working/runs/detect/train/train_batch1801.jpg
/kaggle/working/runs/detect/train/train_batch0.jpg
/kaggle/working/runs/detect/train/weights/
/kaggle/working/runs/detect/train/weights/best.pt
/kaggle/working/runs/detect/train/weights/last.pt
/kaggle/working/runs/detect/train/weights/last.onnx
/kaggle/working/runs/detect/train/weights/best.onnx
/kaggle/working/runs/detect/train/P_curve.png
/kaggle/working/runs/detect/train/val_batch2_pred.jpg
/kaggle/working/runs/detect/train/results.csv
/kaggle/working/runs/detect/train/results.png
/kaggle/working/runs/detect/train/events.out.tfevents.1716480732.592c85053387.107.0
/kaggle/working/runs/detect/train/labels.jpg
/kaggle/working/runs/detect/train/train_batch2.jpg
/kaggle/working/runs/detect/train/PR_curve.png
/kaggle/working/runs/detect/train/val_batch1_pr